In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Ukoly
1. Prevest data do pandas dataframu
    1. Data jsou ve formatu arff 
2. Navrhnout model a vyhodnotim model

In [90]:
from scipy.io import arff
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn

In [91]:
path_arff_train = "/kaggle/input/heart-beat-classification/AbnormalHeartbeat_TRAIN.arff"
path_ts_train = "/kaggle/input/heart-beat-classification/AbnormalHeartbeat_TRAIN.ts"
path_arff_test = "/kaggle/input/heart-beat-classification/AbnormalHeartbeat_TEST.arff"
path_ts_test = "/kaggle/input/heart-beat-classification/AbnormalHeartbeat_TEST.ts"

In [92]:
data_arff_train = arff.loadarff(path_arff_train)
data_arff_test = arff.loadarff(path_arff_test)

Inspekce dat

Data jsou ve formatu arff. Pro pouziti je nutne je nejprve prevest do numpy pole, ktere je nasledne prevedeno do pandas dataframu.
data_arff_train jsou ve formatu touple kde, index 0 obsahuje data a index 1 obsahuje metadata *scipy.io.arff.arffread.MetaData*. Meta data rikaji jakeho typu jsou jednotlive zaznamy. 

Podle zadani maji data frekvenci 4KHz.

In [275]:
data_train = pd.DataFrame(data_arff_train[0])
X_train = data_train.iloc[:, :-1]
y_train = data_train.target
data_test = pd.DataFrame(data_arff_test[0])
X_test = data_test.iloc[:, :-1]
y_test = data_test.target

In [276]:
X_test.shape

(205, 18530)

# PyTorch kod

In [277]:
X_train = torch.tensor(X_train.values, dtype=torch.float32)
y_train = y_train.apply(lambda x: 1 if x == b'Abnormal' else 0)
y_train = torch.tensor(y_train.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_test = y_test.apply(lambda x: 1 if x == b'Abnormal' else 0)
y_test = torch.tensor(y_test.values, dtype=torch.float32)

In [278]:
class my_dataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __getitem__(self, i):
        return self.X[i], self.y[i]
    
    def __len__(self):
        return len(self.X)

In [279]:
train_dataset = my_dataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=51, shuffle=True)
test_dataset = my_dataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=45, shuffle=False)

Prehled dat

In [280]:
x, targets = next(iter(train_dataloader))
print(x[0], targets[0])

tensor([-0.0100, -0.0206, -0.0161,  ..., -0.0082, -0.0070, -0.0060]) tensor(1.)


In [281]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(18530, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Softmax(dim=1),
        )

    def forward(self, x):
        return self.linear_relu_stack(x)
    
    

In [303]:
learning_rate = 1e-3
batch_size = 8
epochs = 100

model = Model()
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [304]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        y = y.view(-1, 1)
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

#train_loop(train_dataloader, model, loss_fn, optimizer)

In [305]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            y = y.view(-1, 1)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

#test_loop(test_dataloader, model, loss_fn)

In [306]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 27.450981  [   51/  204]
Test Error: 
 Accuracy: 73.2%, Avg loss: 33.333333 

Epoch 2
-------------------------------
loss: 25.490196  [   51/  204]
Test Error: 
 Accuracy: 73.2%, Avg loss: 33.333333 

Epoch 3
-------------------------------
loss: 27.450981  [   51/  204]
Test Error: 
 Accuracy: 73.2%, Avg loss: 33.333333 

Epoch 4
-------------------------------
loss: 23.529411  [   51/  204]
Test Error: 
 Accuracy: 73.2%, Avg loss: 33.333333 

Epoch 5
-------------------------------
loss: 27.450981  [   51/  204]
Test Error: 
 Accuracy: 73.2%, Avg loss: 33.333333 

Epoch 6
-------------------------------
loss: 27.450981  [   51/  204]
Test Error: 
 Accuracy: 73.2%, Avg loss: 33.333333 

Epoch 7
-------------------------------
loss: 21.568628  [   51/  204]
Test Error: 
 Accuracy: 73.2%, Avg loss: 33.333333 

Epoch 8
-------------------------------
loss: 27.450981  [   51/  204]
Test Error: 
 Accuracy: 73.2%, Avg loss: 33.333333 

Epoch 9
